In [1]:
import tkinter as tk # Tkinter will create a file dialog screen to obtain directory with all info
import os
import cv2
from tkinter import filedialog
from skimage import io
from skimage.registration import phase_cross_correlation
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import time
from matplotlib.collections import EllipseCollection

In [2]:
def convert_points_to_movements(points, x_tip, y_tip):
    # Find the closest point to the tip location
    
    points = points.to_numpy()
    
    firstpoint = points[0]
    lastpoint = points[-1]
    
    dist_to_start = (firstpoint[0]-x_tip)**2 + (firstpoint[1]-y_tip)**2
    dist_to_stop = (lastpoint[0]-x_tip)**2 + (lastpoint[1]-y_tip)**2
    # If we're starting off closer to the stop, then do the trace in reverse
    if dist_to_stop < dist_to_start:
        points = np.flip(points, axis=0) 

    # Convert from a list of points to the list of moves
    x = [(points[i][0] - points[i+1][0]) for i in range(len(points)-1)]
    y = [(points[i][1] - points[i+1][1]) for i in range(len(points)-1)]

    # Finally, add on the movement necessary to reach the first point using the position of the tip
    firstmove_x = x_tip - points[0][0]
    firstmove_y = y_tip - points[0][1]

    x = [firstmove_x] + x
    y = [firstmove_y] + y

    # Convert datatypes to ints
    x = [int(i) for i in x]
    y = [int(i) for i in y]
    
    return x,y

In [ ]:
while 1:
    counter = 0
    breaker = 0
    
    # Before the directories are initialized, just wait
    while os.path.isdir(r'C:\Users\cstashko\Desktop\AFM_expt_temp\pics'):
        pics_old = os.listdir('C:/Users/cstashko/Desktop/AFM_expt_temp/pics/')
        num_pics = len(pics_old)
        num_pics_old = num_pics

        # Wait for the points file
        while not os.path.isfile(r'C:\Users\cstashko\Desktop\AFM_expt_temp\points.txt'):
            time.sleep(1)
            # After the last path, the pics directory will be moved to the save directory.
            # If that happens, then we need to break out of not just this while loop, but the higher level while loop as well
            # This will allow us to change pics_old back to an empty list before returning to this point
            if not os.path.isdir(r'C:\Users\cstashko\Desktop\AFM_expt_temp\pics'):
                breaker = 1;
                break
        if breaker == 1:
            break

        print('tip location registered')
        # Read the points and tip locations, then format the movements
        points = pd.read_csv(r'C:\Users\cstashko\Desktop\AFM_expt_temp\points.txt', header=None, sep=' ')
        tip_location = pd.read_csv(r'C:\Users\cstashko\Desktop\AFM_expt_temp\tip_location.txt', sep=' ', header=None)
        x_tip = tip_location[0].tolist()[0]
        y_tip = tip_location[1].tolist()[0]
        x,y = convert_points_to_movements(points, x_tip, y_tip)

        # Wait for the first pic to be saved
        while(num_pics == num_pics_old):
            num_pics = len(os.listdir('C:/Users/cstashko/Desktop/AFM_expt_temp/pics/'))
            time.sleep(1)
        pics_new = os.listdir('C:/Users/cstashko/Desktop/AFM_expt_temp/pics/')
        pic_old = np.setdiff1d(pics_new, pics_old)[0]
        pic_old = io.imread('C:/Users/cstashko/Desktop/AFM_expt_temp/pics/'+pic_old)
        pic_old = np.asarray(pic_old)
        pics_old = pics_new
        num_pics_old = num_pics

        #####
        #####
        ##### Gradient Descent feedback loop
        #####
        #####
        # Write the first movement
        i = 0
        x_0 = x[i]
        y_0 = y[i]
        with open("C:/Users/cstashko/Desktop/AFM_expt_temp/in/test.txt", 'a') as f:
            f.write('0 '+ str(x_0) + ' ' + str(y_0) + ' ' + str(counter))
            f.close()

            
        while i < len(x):

            # Wait until an additional picture has been taken
            while(num_pics == num_pics_old):
                pics_new = os.listdir('C:/Users/cstashko/Desktop/AFM_expt_temp/pics/')
                num_pics = len(pics_new)
                time.sleep(1)
                counter += 1

            # Update the number of files
            num_pics_old = num_pics

            # Import the new picture
            pic_new = np.setdiff1d(pics_new, pics_old)[0]
            pic_new = io.imread('C:/Users/cstashko/Desktop/AFM_expt_temp/pics/'+pic_new)
            pic_new = np.asarray(pic_new)

            # Once we have the new pic, put it in the memory so we can move on to the next one
            pics_old = pics_new

            then = time.time()
            move = phase_cross_correlation(pic_old, pic_new)[0]
            x_move = int(move[1])
            y_move = int(move[0])

            now = time.time()
            print(now-then)

            # Reverse the sign on x_move and y_move
            x_move = x_move*-1
            y_move = y_move*-1

            # Update the previous picture
            pic_old = pic_new


            # If the x_move, y_move is within some tolerance of the specified point, then now it's time to poke.
            # Is the point within 5 um?
            # Note that we're adding instead ofsubstracting because the direction gets flipped
            if (x_0 - x_move)**2 + (y_0 - y_move)**2 < 36:
                x_residual = x_0 - x_move
                y_residual = y_0 - y_move

                print('close enough')
                with open("C:/Users/cstashko/Desktop/AFM_expt_temp/in/test.txt", 'a') as f:
                    f.write('1 0 0 '+str(counter))
                    f.close()
                # Now get the next point into x_0, y_0
                i += 1

                # Stop if it's the last point
                if i == len(x):
                    break
                # Otherwise, get the next move
                x_0 = x[i] + x_residual
                y_0 = y[i] + y_residual
                print('Next move is x=' + str(x_0) + ' and y=' + str(y_0))

            # Otherwise, make another movement
            else:
                x_0 = x_0-x_move
                y_0 = y_0-y_move
                print('x_0 is '+str(x_0)+' and y_0 is '+str(y_0))
                with open("C:/Users/cstashko/Desktop/AFM_expt_temp/in/test.txt", 'a') as f:
                    f.write('0 '+ str(x_0) + ' ' + str(y_0) + ' ' + str(counter))
                    f.close()

        '''# Wait for the final forceplot to be taken
        while(num_pics == num_pics_old):
            pics_new = os.listdir('C:/Users/cstashko/Desktop/AFM_expt_temp/pics/')
            num_pics = len(pics_new)
            time.sleep(1)
            counter += 1
        '''
        
        # Remove the points file to prepare for the next one
        os.remove(r'C:\Users\cstashko\Desktop\AFM_expt_temp\points.txt')

tip location registered
0.4000227451324463
x_0 is 13 and y_0 is -7
0.38102149963378906
x_0 is 9 and y_0 is -5
0.3820216655731201
x_0 is 7 and y_0 is -2
0.37702131271362305
close enough
Next move is x=81 and y=62
0.38302183151245117
x_0 is 81 and y_0 is 62
0.3740212917327881
x_0 is 28 and y_0 is 15
0.3820219039916992
x_0 is 8 and y_0 is -8
0.37702155113220215
x_0 is 6 and y_0 is -10
0.3760213851928711
x_0 is 3 and y_0 is -12
0.3800215721130371
x_0 is 1 and y_0 is -9
0.3800218105316162
close enough
Next move is x=77 and y=61
0.3780217170715332
x_0 is 77 and y_0 is 61
0.38302159309387207
x_0 is -29 and y_0 is 40
0.3870220184326172
x_0 is -26 and y_0 is 13
0.4090232849121094
x_0 is -17 and y_0 is 12
0.37902164459228516
x_0 is -10 and y_0 is 1
0.38302183151245117
x_0 is -6 and y_0 is 3
0.3780214786529541
close enough
Next move is x=64 and y=71
0.37902188301086426
x_0 is 64 and y_0 is 71
0.3780217170715332
x_0 is 30 and y_0 is -27
0.37702155113220215
x_0 is 16 and y_0 is -34
0.37602138519287

0.3850221633911133
x_0 is -48 and y_0 is -4
0.3870220184326172
close enough
Next move is x=-44 and y=-4
0.37902140617370605
x_0 is -44 and y_0 is -4
0.38102197647094727
close enough
Next move is x=-44 and y=-4
0.38102197647094727
x_0 is -44 and y_0 is -4
0.3870220184326172
x_0 is 7 and y_0 is -1
0.3830220699310303
x_0 is 6 and y_0 is -2
0.3840219974517822
close enough
Next move is x=-44 and y=-6
0.38102173805236816
x_0 is -44 and y_0 is -6
0.3840219974517822
x_0 is -6 and y_0 is 1
0.3760213851928711
close enough
Next move is x=-50 and y=-2
0.38302183151245117
x_0 is -50 and y_0 is -2
0.38102149963378906
close enough
Next move is x=-45 and y=-4
0.3780214786529541
x_0 is -45 and y_0 is -4
0.3850219249725342
close enough
Next move is x=-51 and y=-9
0.38602232933044434
x_0 is -51 and y_0 is -9
0.37702155113220215
close enough
Next move is x=-44 and y=-16
0.37902140617370605
x_0 is -44 and y_0 is -16
0.3760213851928711
close enough
Next move is x=-42 and y=-5
0.3840219974517822
x_0 is -42 a

0.3840219974517822
x_0 is -7 and y_0 is 5
0.37702155113220215
close enough
Next move is x=30 and y=-15
0.37902164459228516
x_0 is 30 and y_0 is -15
0.3740212917327881
x_0 is 24 and y_0 is -27
0.38102149963378906
x_0 is 15 and y_0 is -29
0.37302160263061523
x_0 is 11 and y_0 is -5
0.3850219249725342
x_0 is 7 and y_0 is -4
0.3820216655731201
close enough
Next move is x=38 and y=-22
0.3780217170715332
x_0 is 38 and y_0 is -22
0.3850219249725342
x_0 is 20 and y_0 is -16
0.3850219249725342
x_0 is 8 and y_0 is -6
0.4090232849121094
close enough
Next move is x=38 and y=-22
0.37902164459228516
x_0 is 38 and y_0 is -22
0.3760213851928711
close enough
Next move is x=34 and y=-22
0.3850219249725342
x_0 is 34 and y_0 is -22
0.3840219974517822
close enough
Next move is x=37 and y=-19
0.3820219039916992
x_0 is 37 and y_0 is -19
0.3820219039916992
x_0 is 1 and y_0 is -6
0.3760216236114502
close enough
Next move is x=38 and y=-13
0.37702155113220215
x_0 is 38 and y_0 is -13
0.3800218105316162
close en

In [ ]:
with open("C:/Users/cstashko/Desktop/AFM_expt_temp/pics/test.txt", 'a') as f:
                    f.write('1 0 0 '+str(counter))
                    f.close()

In [ ]:
pic_new.shape